In [ ]:
from selenium import webdriver
# By : Tag에 접근하기 편하게 하는 기능
from selenium.webdriver.common.by import By
# Keys : 키보드의 이벤트
from selenium.webdriver.common.keys import Keys
import pandas as pd 
from sqlalchemy import create_engine

In [ ]:
# 웹드라이버를 이용하여 구글크롬 브라우져를 오픈 
driver =webdriver.Chrome()

In [ ]:
# 특정 주소로 요청을 보낸다. 
driver.get('http://www.naver.com')

In [ ]:
# Tag 중 id가 query인 태그를 선택하여 변수에 저장 
search_element = \
    driver.find_element(By.ID, 'query')

In [ ]:
type(search_element)

In [ ]:
# search_element(검색어창)에 특정 텍스트를 
# 입력(데이터를 보낸다)한다. 
search_element.send_keys('아이폰')

In [ ]:
# search_element에서 ENTER 이벤트를 발생
search_element.send_keys(Keys.ENTER)

In [ ]:
# 쇼핑 버튼(Tag)을 선택한다. 
# LINK_TEXT가 쇼핑인 태그 -> 
# 해당 조건에 맞는 태그가 몇개인가?
len(driver.find_elements(By.LINK_TEXT, '쇼핑'))
# 조건에 맞는 태그의 개수가 1개 이므로 클릭 이벤트 발생
driver.find_element(By.LINK_TEXT, '쇼핑').click()

In [ ]:
# 쇼핑 페이지에서 검색어의 결과(물건의 가격과 제품명)를 
# 크롤링 -> 해당 페이지의 소스코드를 불러온다. 
html_data = driver.page_source

In [ ]:
# bs4 라이브러리 안에 있는 BeautifulSoup을 이용하여 
# html_data를 parsing(데이터의 타입을 변환)
from bs4 import BeautifulSoup as bs

In [ ]:
soup = bs(html_data, 'html.parser')

In [ ]:
# html_data가 2개의 탭중 어떤 소스코드인지 확인 
# 탭의 제목이 다르기 때문에 title Tag 확인
# 특정 태그를 확인 
soup.title

In [ ]:
# page_source가 첫번째 탭인것을 확인 
# 두번째 탭으로 page_source를 이동 
# 탭의 정보를 확인
# driver에서 탭의 주소를 확인 
driver.window_handles

In [ ]:
# driver에서 탭을 이동
driver.switch_to.window(
    driver.window_handles[1]
)

In [ ]:
html_data2 = driver.page_source
soup2 = bs(html_data2, 'html.parser')

soup2.title

#### 네이버 쇼핑 크롤링
1. div 태그중 id가 'content'인 태그를 선택하여 저장(content_data)
2. 저장한 태그(content_data)에서 div 태그중 class가 'product_item__KQayS' 인 태그 정보를 모두 저장 (div_list)
3. div_list에서 div 태그중 class가 'product_title__ljFM_'인 태그의 문자를 추출(상품명)
4. div_list에서 span 태그중 class가 'price'인 태그의 문자를 추출(상품 격)

In [ ]:
content_data = soup2.find(
    'div', 
    attrs={
        'id' : 'content'
    }
)

In [29]:
import re

In [30]:
# content_data에서 div중 class가 'product_item_KQayS'인 태그의 개수를 확인
# class의 이름에 'product_item'이 포함되어있는 태그
len(
    content_data.find_all(
        'div', 
        # attrs={
        #     'class' : 'product_item__KQayS'
        # }
        attrs = {
            'class' : re.compile('product_item')
        }
    )
)

3

In [31]:
div_list = content_data.find_all(
    'div', 
    # attrs={
    #     'class' : 'product_item__KQayS'
    #     }
    attrs = {
            'class' : re.compile('product_item')
    }
)

In [ ]:
div_data = div_list[0]

In [35]:
item_name = div_data.find(
    'div', 
    # attrs = {
    #     'class' : 'product_title__ljFM_'
    # }
    attrs = {
        'class' : re.compile('product_title')
    }
).get_text()

In [36]:
item_price = div_data.find('span', attrs = {'class' : 'price'}).get_text()

In [34]:
# 상품의 이름과 상품의 가격을 dict 형태로 생성

# 비어있는 dict 생성 
dict_data = {}

In [38]:
# dict에 데이터를 추가 
dict_data['상품명'] = item_name
dict_data['가격'] = item_price

In [39]:
dict_data

{'상품명': 'Apple 애플 아이폰 13 미니 128GB 새상품', '가격': '298,000원'}

In [64]:
values = []
for div_data in div_list:
    item_name = div_data.find(
        'div', 
        attrs = {
            'class' : re.compile('product_title')
        }
    ).get_text()
    item_price = div_data.find('span', attrs = {'class' : 'price'}).get_text()
    item_url = div_data.find('a')['href']
    dict_data = {
        '상품명' : item_name, 
        '가격' : item_price, 
        'url' : item_url
    }
    values.append(dict_data)

In [65]:
values

[{'상품명': 'Apple 애플 아이폰 13 미니 128GB 새상품',
  '가격': '298,000원',
  'url': 'https://cr.shopping.naver.com/adcr?x=1zLjF9l0wn2sDaNEbXMa4f%2F%2F%2Fw%3D%3DsBY7ovv7PqUbNeXIcdqvSfK1odJ2%2F6LBtogIwFjN3ri1yoYErd7Wx1dfdVO6hh7qVWl9Lw2a9XdaIHIXvhymRpMKdlAuo4CRofAy%2FiHp8jqv8RtWXQ%2B4yWv%2FzzZujOpojSOJcOlgKXJrBbuiphAOYksvvFzydFsJfvMWJZSW9JRsX03%2FPXqL6DRioMWujJoazyaxxRm4H8HcYDi0B3B3iQmfQT7uWwvxdz8nMvhXCKF1zPnBo0jGi%2FSdSlDWdPE2TZznAAH%2FDJkgOxgHmnSf1jZwCn%2FAtvdQblXmaS8p1DhgQhkkHXK4%2FmSRy5icYE%2F27ZC%2BSXUxwyLUckntucAABtftfixCF1WpHtYmpQGaH5CMUlnTe%2Bq%2FRuRxi5rXKKROMbF4OZVL2Q%2FcwuxYvViSTh%2F4PzmWxMBgw%2FsYqXl1ymjHUl1Mb6CmDxR1AktcPJbKngLwgHSP3DD1GBfo2Ry7n82dFqn9EtHPPkp8WxjFFMRTBK7bxfhjLutrsEfkVCvcrmizsw57iASTSdiNeVn6LnWD7VD1bWrOzy%2FEGoKY7dKPTn%2BIvcxawVxNuePP9ktetRpdLJBo4KrDhyu8x%2FXb0ZktX%2FDLZQdSo3ix0kwlBIJeB%2FaZ8wxZd28c0hrtmjZHpzY6of2Xpggjm6A4Z8HgJhg%3D%3D&nvMid=88573763568&catId=50001519'},
 {'상품명': '아이폰 16 프로 128GB [자급제]',
  '가격': '최저1,538,900원',
  'url': 'https://cr.shopping.naver.com/adcr?x=7

In [42]:
# div_list에서 div 태그중 class의 이름에 'product_img_area'가 포함된 태그를 찾는다. 
# 해당 태그에서 img 태그에 있는 src 속성의 값을 추출
div_data = div_list[0]

In [63]:
# div_data에서 a태그(하이퍼링크) 첫번째 정보를 확인하여 href 속성의 값을 출력
# html문서에서 첫번째 태그의 정보를 확인 
    # htmldata.태그명
    # htmldata.find(태그명)
# 특정 태그의 속성의 값을 확인 
    # Tag[속성명]
# div_data.a['href']
div_data.find('a')['href']

'https://cr.shopping.naver.com/adcr?x=1zLjF9l0wn2sDaNEbXMa4f%2F%2F%2Fw%3D%3DsBY7ovv7PqUbNeXIcdqvSfK1odJ2%2F6LBtogIwFjN3ri1yoYErd7Wx1dfdVO6hh7qVWl9Lw2a9XdaIHIXvhymRpMKdlAuo4CRofAy%2FiHp8jqv8RtWXQ%2B4yWv%2FzzZujOpojSOJcOlgKXJrBbuiphAOYksvvFzydFsJfvMWJZSW9JRsX03%2FPXqL6DRioMWujJoazyaxxRm4H8HcYDi0B3B3iQmfQT7uWwvxdz8nMvhXCKF1zPnBo0jGi%2FSdSlDWdPE2TZznAAH%2FDJkgOxgHmnSf1jZwCn%2FAtvdQblXmaS8p1DhgQhkkHXK4%2FmSRy5icYE%2F27ZC%2BSXUxwyLUckntucAABtftfixCF1WpHtYmpQGaH5CMUlnTe%2Bq%2FRuRxi5rXKKROMbF4OZVL2Q%2FcwuxYvViSTh%2F4PzmWxMBgw%2FsYqXl1ymjHUl1Mb6CmDxR1AktcPJbKngLwgHSP3DD1GBfo2Ry7n82dFqn9EtHPPkp8WxjFFMRTBK7bxfhjLutrsEfkVCvcrmizsw57iASTSdiNeVn6LnWD7VD1bWrOzy%2FEGoKY7dKPTn%2BIvcxawVxNuePP9ktetRpdLJBo4KrDhyu8x%2FXb0ZktX%2FDLZQdSo3ix0kwlBIJeB%2FaZ8wxZd28c0hrtmjZHpzY6of2Xpggjm6A4Z8HgJhg%3D%3D&nvMid=88573763568&catId=50001519'

In [ ]:
# driver 스크롤을 가장 밑으로 내린다. 
# driver.execute_script(
#     "window.scrollTo(0, document.body.scrollHeight);"
# )

In [77]:
# driver에서 스크롤을 일정 간격으로 내린다. 
driver.execute_script(
    "window.scrollBy(0, 800);"
)

In [78]:
html_data3 = driver.page_source

In [80]:
# BeautifulSoup을 이용해서 데이터 파싱 
soup3 = bs(html_data3, 'html.parser')
# 광고 상품을 제외한 모든 상품의 이름과 가격 링크 주소를 
# 2차원 데이터(리스트 안에 딕셔너리)로 생성
content_data = soup3.find('div', attrs={'id' : 'content'})
div_list = content_data.find_all('div', 
    attrs = {
            'class' : re.compile('product_item')
    }
)
values = []
for div_data in div_list:
    item_name = div_data.find('div', 
        attrs = {'class' : re.compile('product_title')}).get_text()
    item_price = div_data.find('span', attrs = {'class' : 'price'}).get_text()
    item_url = div_data.find('a')['href']
    dict_data = {
        '상품명' : item_name, 
        '가격' : item_price, 
        'url' : item_url
    }
    values.append(dict_data)
# 생성된 2차원 데이터를 데이터프레임으로 생성 
df = pd.DataFrame(values)
# df
# 검색어('아이폰')를 파일명으로 csv 파일을 생성하고 
df.to_csv('아이폰.csv', index=False)
# 인덱스는 제외한다.

In [83]:
# driver를 종료 
driver.quit()